In [2]:
import pandas as pd
import dask.dataframe as dd

df = dd.read_csv("C:/Users/myaka/Desktop/Final_Year_project/data/file.csv")

In [3]:
print(df.head())


                         id                 date     user       pc  \
0  {F3E2-X3MV05YQ-3516SZDT}  01/02/2010 07:19:41  SDH2394  PC-5849   
1  {I6N1-Z7VL92UY-8715ESKQ}  01/02/2010 07:21:30  SDH2394  PC-5849   
2  {G4X5-J7MH70FV-8936QVSB}  01/02/2010 07:22:11  SDH2394  PC-5849   
3  {M2M7-Z5ST21EU-6704NSKO}  01/02/2010 07:24:06  SDH2394  PC-5849   
4  {R0A9-O9XB25PE-9236MALV}  01/02/2010 07:24:45  SDH2394  PC-5849   

                  filename    activity  to_removable_media  \
0          R:\60WBQE7S.doc   File Open               False   
1          R:\0VGILDW8.pdf  File Write                True   
2          R:\60WBQE7S.doc   File Copy               False   
3  R:\22B5gX4\H8Y96RRE.doc  File Write                True   
4  R:\SDH2394\7XRCV2N5.pdf   File Copy                True   

   from_removable_media                                            content  
0                  True  D0-CF-11-E0-A1-B1-1A-E1 Ernesztin's brother, L...  
1                 False  25-50-44-46-2D ---- Bengali

In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest

# Ensure 'date' column is parsed as datetime
df['date'] = dd.to_datetime(df['date'], errors='coerce')


df['activity_new'] = df['activity'].map_partitions(lambda x: x.astype('category').cat.codes, meta=('activity', 'int64'))


# Encode boolean features
encoder = LabelEncoder()
df['to_removable_media_new'] = df['to_removable_media'].map_partitions(encoder.fit_transform, meta=('to_removable_media', 'int64'))
df['from_removable_media_new'] = df['from_removable_media'].map_partitions(encoder.fit_transform, meta=('from_removable_media', 'int64'))

# Extract datetime features
df['hour'] = df['date'].dt.hour

df_new = df[['activity_new', 'to_removable_media_new', 'from_removable_media_new', 'hour']]

# Convert Dask dataframe to Pandas for model training
df_new = df_new.compute()

# Train Isolation Forest for anomaly detection
iso_forest = IsolationForest(contamination=0.05, random_state=42)
df_new['anomaly_score'] = iso_forest.fit_predict(df_new)

# Detect anomalies
df_new['is_anomaly'] = df_new['anomaly_score'].apply(lambda x: 1 if x == -1 else 0)

# Save results





In [8]:
print(df_new[df_new['is_anomaly'] == 1])  # Print only anomalous rows


        activity_new  to_removable_media_new  from_removable_media_new  hour  \
0                  2                       0                         1     7   
1                  3                       1                         0     7   
3                  3                       1                         0     7   
4                  0                       1                         0     7   
6                  3                       1                         0     7   
...              ...                     ...                       ...   ...   
101330             1                       0                         1    23   
101331             2                       0                         1    23   
101332             2                       0                         1    23   
101333             2                       0                         1    23   
101334             1                       0                         1    23   

        anomaly_score  is_anomaly  
0  

In [11]:
df_graph = df.merge(df_new[['anomaly_score', 'is_anomaly']], left_index=True, right_index=True)
print(df_graph.head())


                         id                date     user       pc  \
0  {F3E2-X3MV05YQ-3516SZDT} 2010-01-02 07:19:41  SDH2394  PC-5849   
0  {F3E2-X3MV05YQ-3516SZDT} 2010-01-02 07:19:41  SDH2394  PC-5849   
0  {F3E2-X3MV05YQ-3516SZDT} 2010-01-02 07:19:41  SDH2394  PC-5849   
0  {F3E2-X3MV05YQ-3516SZDT} 2010-01-02 07:19:41  SDH2394  PC-5849   
0  {F3E2-X3MV05YQ-3516SZDT} 2010-01-02 07:19:41  SDH2394  PC-5849   

          filename   activity  to_removable_media  from_removable_media  \
0  R:\60WBQE7S.doc  File Open               False                  True   
0  R:\60WBQE7S.doc  File Open               False                  True   
0  R:\60WBQE7S.doc  File Open               False                  True   
0  R:\60WBQE7S.doc  File Open               False                  True   
0  R:\60WBQE7S.doc  File Open               False                  True   

                                             content  activity_new  \
0  D0-CF-11-E0-A1-B1-1A-E1 Ernesztin's brother, L...            

In [16]:
import pickle

with open("anomaly_file.pkl", "wb") as model_file:
    pickle.dump(iso_forest, model_file)